In [ ]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WrapperConfig, SMOL_LM_135M

In [6]:
cfg = WrapperConfig(
    model_id_or_path=SMOL_LM_135M,
    max_samples=1000,
    n_epochs=10,
    data_module_choice="ultra_feedback",
    notebook_mode=True,
    tuning_mode="full",
    eval_data_mode="fixed"
)

wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2024-12-19 17:01:10.195 | INFO     | trl_wrapper.trainer_wrapper:init_model:120 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct


In [7]:
wrapper.init_data_module()

2024-12-19 17:01:14.085 | INFO     | model.utils:setup:102 - Loading dataset for stage fit
2024-12-19 17:01:14.086 | INFO     | model.utils:setup:104 - Processing dataset for stage fit, workers: 1, cache dir dataset_caches/ultrafeedback
2024-12-19 17:01:15.293 | INFO     | dataset.squad:load_dataset:246 - Loaded dataset with 60917 samples
2024-12-19 17:01:15.346 | WARNING  | model.utils:setup:145 - Computed columns not found in dataset: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'] assuming using 


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [8]:
wrapper.init_trainer()

2024-12-19 17:01:17.880 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:226 - Initializing DPOtrainer, run: run-882269, project: codecontests-llama-3b
2024-12-19 17:01:17.881 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:229 - logprobs cache location: dataset_caches/ultrafeedback/b68b303b/ref_logprobs_cache peft config: False
2024-12-19 17:01:17.881 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:230 - WrapperConfig(model_id_or_path='HuggingFaceTB/SmolLM2-135M-Instruct', notebook_mode=True, max_sequence_length=2048, prompt_length=1024, max_eval_sample_length=1024, max_samples=1000, train_batch_size=4, eval_batch_size=2, gradient_accumulation_steps=1, gradient_checkpointing=False, root_dir=None, data_module_choice='ultra_feedback', wandb_project_name='codecontests-llama-3b', n_epochs=10, max_eval_dataset_size=None, adapter_path=None, dpo_beta=0.1, learning_rate=5e-05, max_grad_norm=0.3, lora_rank=512, lora_dropout=0.2, lora_alpha=256, logprob_precompute_batch_size=16

In [13]:
wrapper.train()

2024-12-19 17:37:41.489 | INFO     | trl_wrapper.trainer_wrapper:train:361 - Starting training.
2024-12-19 17:37:41.882 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:143 - Generating samples...
2024-12-19 17:37:41.883 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:61 - Generating policy samples, max length: 1024...
2024-12-19 17:37:46.886 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:79 - Generating reference samples, max length: 1024...
2024-12-19 17:38:06.926 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:145 - Generated samples.
2024-12-19 17:38:07.135 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:216 - Saved eval samples.


┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ policy                                             │ ref                                                │ chosen                                             │ rejected                                           │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ systemYou are Dolphin, a helpful AI                │ I'm ready to play the "Retirement Roulette" game.  │ I'm ready to play the "Retirement Roulette" game.  │ I'm definitely ready for the challenge

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,No log,0.693147,0.000000,0.000000,0.000000,0.000000,-598.114319,-423.071014,3.299754,3.410845
100,0.693100,0.693147,0.000000,0.000000,0.000000,0.000000,-598.318420,-423.303833,3.310116,3.422456
200,0.693100,0.693147,0.000000,0.000000,0.000000,0.000000,-598.740479,-425.604523,3.518910,3.628979
300,0.693100,0.693147,0.000000,0.000000,0.000000,0.000000,-603.346619,-433.112885,3.680444,3.789001


2024-12-19 17:38:41.043 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:143 - Generating samples...
2024-12-19 17:38:41.043 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:61 - Generating policy samples, max length: 1024...
2024-12-19 17:38:46.082 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:79 - Generating reference samples, max length: 1024...
2024-12-19 17:39:06.145 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:145 - Generated samples.
2024-12-19 17:39:06.175 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:216 - Saved eval samples.


┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ policy                                             │ ref                                                │ chosen                                             │ rejected                                           │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ systemYou are Dolphin, a helpful AI                │ I'm ready to play the "Retirement Roulette" game.  │ I'm ready to play the "Retirement Roulette" game.  │ I'm definitely ready for the challenge

...

┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ policy                                             │ ref                                                │ chosen                                             │ rejected                                           │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ systemYou are Dolphin, a helpful AI                │ I'm ready to play the "Retirement Roulette" game.  │ I'm ready to play the "Retirement Roulette" game.  │ I'm definitely ready for the challenge

2024-12-19 17:40:38.165 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:143 - Generating samples...
2024-12-19 17:40:38.166 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:61 - Generating policy samples, max length: 1024...
2024-12-19 17:40:43.203 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:79 - Generating reference samples, max length: 1024...
2024-12-19 17:41:03.231 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:145 - Generated samples.
2024-12-19 17:41:03.266 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:216 - Saved eval samples.


┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ policy                                             │ ref                                                │ chosen                                             │ rejected                                           │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ systemYou are Dolphin, a helpful AI                │ Retirement Roulette is a thrilling game of chance  │ Retirement Roulette is a thrilling game of chance  │ I'm definitely ready for the challenge

2024-12-19 17:41:36.476 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:143 - Generating samples...
2024-12-19 17:41:36.477 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:61 - Generating policy samples, max length: 1024...
2024-12-19 17:41:41.503 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:79 - Generating reference samples, max length: 1024...


KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
for prompt, chosen, rejected, logprob_diff in []:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))